In [1]:
# `ibis` a pythonic query language.

    import ibis.omniscidb, dask, intake, sqlalchemy, pandas, pyarrow as arrow, altair, h5py as hdf5

<!--    
    
    import refs
    reload(refs)
    from refs import __doc__ as refs
    over_time = pandas.read_csv('annual_tags.csv').set_index('year').fillna('')
    
-->

# `ibis` a pythonic query language.

    import ibis.omniscidb, dask, intake, sqlalchemy, pandas, pyarrow as arrow, altair, h5py as hdf5

<!--    
    
    import refs
    reload(refs)
    from refs import __doc__ as refs
    over_time = pandas.read_csv('annual_tags.csv').set_index('year').fillna('')
    
-->

In [2]:
The design of `ibis`, and other open source projects like `intake` and `dask`, begin with the pioneering 
`"blaze"` ecosystem provided interfaces to _**store, describe, query, and process**_ data.
"blaze" had ambitious goals and is now defunct, but its influences resonate throughout
the scientific python community development. 

`dask` emerged directly from `"blaze"`; meanwhile
contributors can now be found working on projects like `ibis` and `arrow`. 
[`ibis`][ibis] took the role of providing a generic query language for data in python, that 
is independent of the computation.

In this document we'll describe the different `ibis` backend architectures and what it means to be independent of compute.

{{refs}}

The design of `ibis`, and other open source projects like `intake` and `dask`, begin with the pioneering 
`"blaze"` ecosystem provided interfaces to _**store, describe, query, and process**_ data.
"blaze" had ambitious goals and is now defunct, but its influences resonate throughout
the scientific python community development. 

`dask` emerged directly from `"blaze"`; meanwhile
contributors can now be found working on projects like `ibis` and `arrow`. 
[`ibis`][ibis] took the role of providing a generic query language for data in python, that 
is independent of the computation.

In this document we'll describe the different `ibis` backend architectures and what it means to be independent of compute.

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [3]:
## The design of [`ibis` backends][backends].

Currently, `ibis` supports __>10__ backends.
    
    >>> dir(ibis)
    [...HDFS...WebHDFS...bigquery...clickhouse...hdf5...impala...omniscidb...pandas...pyspark...spark...sql...sqlite...]
    
A backend takes an `ibis` query expression and applies computation, and the query is independent of the computation.
A backend implementation, that can be queried with `ibis`, has one of the three following architectures.

1. Direct execution backends - `pandas and hdf5`. 
2. Expression generating backends that create `sqlalchemy` expressions.
3. String generating backends - `ibis.bigquery and ibis.omniscidb`

In the next few sections we'll unravel some of the different capiabilities of each approach.

{{refs}}

## The design of [`ibis` backends][backends].

Currently, `ibis` supports __>10__ backends.
    
    >>> dir(ibis)
    [...HDFS...WebHDFS...bigquery...clickhouse...hdf5...impala...omniscidb...pandas...pyspark...spark...sql...sqlite...]
    
A backend takes an `ibis` query expression and applies computation, and the query is independent of the computation.
A backend implementation, that can be queried with `ibis`, has one of the three following architectures.

1. Direct execution backends - `pandas and hdf5`. 
2. Expression generating backends that create `sqlalchemy` expressions.
3. String generating backends - `ibis.bigquery and ibis.omniscidb`

In the next few sections we'll unravel some of the different capiabilities of each approach.

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [4]:
## A data-driven history of `ibis` compatability.

The table below looks at over 2000 issues in the ibis project.
It provides an annual looked at the issues tagged in `ibis`
for different backends over __{{len(over_time)}}__ years.

{{over_time.to_html()}}

We note an early focus `ibis.sqlite, sqlalchemy and ibis.impala`. 
Later, work began on the `pandas` backend rounding out the three different types of backgrounds.
From this point, improvements were made to these key backends as `ibis.clickhouse, ibis.spark and "postgres"`. Recently, 
For the past 3 years, Quansight, in partnership with OmniSci, added the `ibis.omniscidb`
string generating backend. Since Quansight Labs has taken on a role as a community maintainer 
for `ibis`. This collaboration introduced geospatial functionality to `ibis`, and we 
have on going efforts to introduce [SQL Server][sql-server] support

> Currently, there is an ongoing effort to a . What backends what you like to
see? `dask`? `altair`?
    
{{refs}}

## A data-driven history of `ibis` compatability.

The table below looks at over 2000 issues in the ibis project.
It provides an annual looked at the issues tagged in `ibis`
for different backends over __6__ years.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>omnisci</th>
      <th>spark</th>
      <th>postgres</th>
      <th>bigquery</th>
      <th>pandas</th>
      <th>sqlite</th>
      <th>impala</th>
      <th>kudu</th>
      <th>geospatial</th>
      <th>clickhouse</th>
      <th>mysql</th>
      <th>sqlalchemy</th>
    </tr>
    <tr>
      <th>year</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2015</th>
      <td></td>
      <td></td>
      <td>2</td>
      <td></td>
      <td>2</td>
      <td>25</td>
      <td>52</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>17</td>
    </tr>
    <tr>
      <th>2016</th>
      <td></td>
      <td></td>
      <td>3</td>
      <td></td>
      <td></td>
      <td>2</td>
      <td>4</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td>3</td>
    </tr>
    <tr>
      <th>2017</th>
      <td></td>
      <td>1</td>
      <td>21</td>
      <td>15</td>
      <td>49</td>
      <td>10</td>
      <td>15</td>
      <td></td>
      <td></td>
      <td>8</td>
      <td></td>
      <td>10</td>
    </tr>
    <tr>
      <th>2018</th>
      <td>31</td>
      <td></td>
      <td>10</td>
      <td>71</td>
      <td>35</td>
      <td>8</td>
      <td>17</td>
      <td></td>
      <td></td>
      <td>9</td>
      <td>2</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2019</th>
      <td>33</td>
      <td>22</td>
      <td>17</td>
      <td>12</td>
      <td>32</td>
      <td>1</td>
      <td>4</td>
      <td></td>
      <td>7</td>
      <td>1</td>
      <td>2</td>
      <td>5</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>38</td>
      <td>3</td>
      <td>4</td>
      <td>2</td>
      <td>4</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>3</td>
      <td>4</td>
      <td>4</td>
      <td></td>
    </tr>
  </tbody>
</table>

We note an early focus `ibis.sqlite, sqlalchemy and ibis.impala`. 
Later, work began on the `pandas` backend rounding out the three different types of backgrounds.
From this point, improvements were made to these key backends as `ibis.clickhouse, ibis.spark and "postgres"`. Recently, 
For the past 3 years, Quansight, in partnership with OmniSci, added the `ibis.omniscidb`
string generating backend. Since Quansight Labs has taken on a role as a community maintainer 
for `ibis`. This collaboration introduced geospatial functionality to `ibis`, and we 
have on going efforts to introduce [SQL Server][sql-server] support

> Currently, there is an ongoing effort to a . What backends what you like to
see? `dask`? `altair`?
    
[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [5]:
> Learn more about working [Geospatial Data in `ibis`][geo-tutorial].

{{refs}}

> Learn more about working [Geospatial Data in `ibis`][geo-tutorial].

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [6]:
## `ibis` direct execution.

`ibis` direct execution backends like `pandas and hdf5` operate on conventional in-memory python objects.
`pandas` is the gold standard for structured data in python, and inspires the api for `ibis`.

    pd = ibis.pandas.connect({'A': pandas.util.testing.makeDataFrame()})
    
`pd` is an `ibis` backend based off `pandas.DataFrame` objects.

    expression = pd.table('A').head()
    
`expression` is an `ibis` query, that has `expression.compile` and `expression.execute` methods.
We'll recognize the __execute__ method when we return `pandas.DataFrame`s from `ibis` expression.
The __compile__ method does not trigger any computation, rather it constructs an intermediate form
that is interpretted by a backend.

    >>> assert isinstance(expression.compile(), ibis.expr.types.TableExpr)
    
In the case of direction execution backends, the `expression` compiles to an the original `ibis` 
expression.  And the computation is carried out based on a set of recipes defined in `ibis`.

In general, we would typically do this work directly in `pandas`, however this work is
practical in mocking tests for expressions independent of backends.

> Learn more about the [HDF5 backend in the tests][test-hdf5].

{{refs}}

/Users/tonyfast/anaconda3/lib/python3.7/site-packages/pandas/util/__init__.py:12: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


## `ibis` direct execution.

`ibis` direct execution backends like `pandas and hdf5` operate on conventional in-memory python objects.
`pandas` is the gold standard for structured data in python, and inspires the api for `ibis`.

    pd = ibis.pandas.connect({'A': pandas.util.testing.makeDataFrame()})
    
`pd` is an `ibis` backend based off `pandas.DataFrame` objects.

    expression = pd.table('A').head()
    
`expression` is an `ibis` query, that has `expression.compile` and `expression.execute` methods.
We'll recognize the __execute__ method when we return `pandas.DataFrame`s from `ibis` expression.
The __compile__ method does not trigger any computation, rather it constructs an intermediate form
that is interpretted by a backend.

    >>> assert isinstance(expression.compile(), ibis.expr.types.TableExpr)
    
In the case of direction execution backends, the `expression` compiles to an the original `ibis` 
expression.  And the computation is carried out based on a set of recipes defined in `ibis`.

In general, we would typically do this work directly in `pandas`, however this work is
practical in mocking tests for expressions independent of backends.

> Learn more about the [HDF5 backend in the tests][test-hdf5].

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [7]:
## `ibis` expression generating backends.

    db = ibis.sqlite.connect('lahmansbaseballdb.sqlite')
    expression = db.table('halloffame').head()
    
Expression generating backends operate on [SQL] databases that interoperator with `sqlalchemy`.

    >>> assert isinstance(expression.compile(), sqlalchemy.sql.Select)
    
In the case of expression generating backends, the intermediate representation is a `sqlalchemy` object.
`sqlalchemy` is _The Database Toolkit for Python_, and `ibis` leverages it compatability
with traditional [SQL] databases.
    
{{refs}}

## `ibis` expression generating backends.

    db = ibis.sqlite.connect('lahmansbaseballdb.sqlite')
    expression = db.table('halloffame').head()
    
Expression generating backends operate on [SQL] databases that interoperator with `sqlalchemy`.

    >>> assert isinstance(expression.compile(), sqlalchemy.sql.Select)
    
In the case of expression generating backends, the intermediate representation is a `sqlalchemy` object.
`sqlalchemy` is _The Database Toolkit for Python_, and `ibis` leverages it compatability
with traditional [SQL] databases.
    
[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [8]:
## `ibis` string generating backends.

```bash
pip install --upgrade ibis-framework[omniscidb]
```

String generating backends allow `ibis` to interface with big data systems that manage 
their own computation. For example, we may connect to an example `omnisci` database.
    
    omnisci = ibis.omniscidb.connect(host='metis.omnisci.com', user='demouser', password='HyperInteractive', port=443, database='omnisci', protocol='https')
    
`omnisci` is described as a string generating backend because the intermediate representation of the
query is a flavor of SQL.

    expression = omnisci.table('upstream_reservoir').head()
    
A string generating expression compiles to `ibis.omniscidb` flavored [SQL], while `ibis.bigquery` may have a different string representatin.
    
    >>> expression.compile()
    'SELECT *\nFROM upstream_reservoir\nLIMIT 5'

    

> Major credit goes to [@xmnlab] in his heroic PR to introduce `ibis.omniscidb` into `ibis`. You can watch
the drama play out in this [Github Issue][omnisci-pr]. If you'd like to learn more about [OmniSci] and
`ibis.omniscidb` checkout the following links.
> * [OmniSci][omnisci]
> * [Quansight Labs- Ibis: Python data analysis productivity framework][labs-post]

{{refs}}

## `ibis` string generating backends.

```bash
pip install --upgrade ibis-framework[omniscidb]
```

String generating backends allow `ibis` to interface with big data systems that manage 
their own computation. For example, we may connect to an example `omnisci` database.
    
    omnisci = ibis.omniscidb.connect(host='metis.omnisci.com', user='demouser', password='HyperInteractive', port=443, database='omnisci', protocol='https')
    
`omnisci` is described as a string generating backend because the intermediate representation of the
query is a flavor of SQL.

    expression = omnisci.table('upstream_reservoir').head()
    
A string generating expression compiles to `ibis.omniscidb` flavored [SQL], while `ibis.bigquery` may have a different string representatin.
    
    >>> expression.compile()
    'SELECT *\nFROM upstream_reservoir\nLIMIT 5'

    

> Major credit goes to [@xmnlab] in his heroic PR to introduce `ibis.omniscidb` into `ibis`. You can watch
the drama play out in this [Github Issue][omnisci-pr]. If you'd like to learn more about [OmniSci] and
`ibis.omniscidb` checkout the following links.
> * [OmniSci][omnisci]
> * [Quansight Labs- Ibis: Python data analysis productivity framework][labs-post]

[ibis]: https://www.ibis-project.org/
[SQL]: https://en.wikipedia.org/wiki/SQL
[Python]: https://en.wikipedia.org/wiki/Python_(programming_language)
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[design]: https://docs.ibis-project.org/design.html
[sqlite]: https://www.sqlite.org/index.html
[pandas]: http://pandas.pydata.org/
[omnisci]: https://www.omnisci.com/
[glue]: https://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html
[dask]: https://dask.org/
[flavor of sql]: https://stackoverflow.com/questions/1326318/difference-between-different-types-of-sql
[dag]: https://en.wikipedia.org/wiki/Directed_acyclic_graph
[data]: http://www.seanlahman.com/baseball-archive/statistics/    
[database connection]: https://en.wikipedia.org/wiki/Database_connection
[tidy data]: https://vita.had.co.nz/papers/tidy-data.pdf
[openteams]: https://openteams.com/
[contributing]: https://docs.ibis-project.org/contributing.html
[qs]: https://www.quansight.com/
[graphviz]: https://graphviz.org
[materialized view]: https://en.wikipedia.org/wiki/Materialized_view
[blaze]: #
[sqlalchemy]: #
[backends]: #
[intake]: #
[arrow]: #
[labs-post]: https://labs.quansight.org/blog/2019/07/ibis-python-data-analysis-productivity-framework/
[geo-tutorial]: https://github.com/ibis-project/ibis/pull/1991
[geo-closed]: https://github.com/ibis-project/ibis/issues?q=label%3Ageospatial+is%3Aclosed
[sql-server]: https://github.com/ibis-project/ibis/pull/1997
[omnisci-pr]: https://github.com/ibis-project/ibis/pull/1419
[test-hdf5]: https://github.com/ibis-project/ibis/blob/master/ibis/file/tests/test_hdf5.py

In [9]:
## Conclusion

We'd like to thank the maintainers of the `ibis` for
their and effort in supporting the `ibis` community.

## Conclusion

We'd like to thank the maintainers of the `ibis` for
their and effort in supporting the `ibis` community.